In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.model_selection import GridSearchCV
from loguru import logger as lg
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

import transformers
import torch
import random

import random
import string
import numpy as np
import unidecode
from unidecode import unidecode
from wordcloud import STOPWORDS
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer, BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, PreTrainedTokenizerFast
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import tensorflow as tf
from tqdm.notebook import tqdm
import numpy as np
import torch
transformers.logging.set_verbosity_debug()


In [13]:
NUM_PROCESSORS = (multiprocessing.cpu_count() - 2)
VERBOSE = 0
DEPARTMENTS_QUANTITY_TO_TRAIN = 6
lg.add("logging/finnlp_bert_large_binary_intra_class_experiment_2.log", rotation="256 MB")
lg.info("\n\n####################################### NEW EXECUTION #######################################\n\n")
lg.info(f"Processors (cores) available: {NUM_PROCESSORS}")

2024-03-01 18:28:27.586 | INFO     | __main__:<module>:6 - 

####################################### NEW EXECUTION #######################################


2024-03-01 18:28:27.588 | INFO     | __main__:<module>:7 - Processors (cores) available: 38


In [14]:
brc = pd.read_csv("Banking_Regulation_Corpora_BRC_anonymous.csv" ,encoding="utf-8", low_memory=False, sep=";")
data_relevant = brc[brc['class'] == 1]
data_irrelevant = brc[brc['class'] == 0]

In [15]:
data_relevant.head(3)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
0,1,DPT_25,2020-12-23,2128,36560,2020-12-23,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,656,67.0,127.0,COMUNICADO,771984,https://www.bcb.gov.br/estabilidadefinanceira/...
1,1,DPT_25,2020-12-28,2129,36569,2020-12-28,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,2110,156.0,322.0,COMUNICADO,772973,https://www.bcb.gov.br/estabilidadefinanceira/...
2,1,DPT_25,2020-12-22,2130,36554,2020-12-22,BACEN,Divulga comunicado do Grupo de Ação Financeira...,120,16.0,18.0,Comunicamos que o Grupo de Ação Financeira con...,792,76.0,127.0,COMUNICADO,771452,https://www.bcb.gov.br/estabilidadefinanceira/...


In [16]:
data_irrelevant.tail(3)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
61861,0,DPT_19,2020-08-12,108110,280,2020-08-10,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,"INSTITUI O PLANO DE CLASSIFICAÇÃO, A TABELA DE...",141,17.0,19.0,O PRESIDENTE DA EMPRESA MUNICIPAL DE INFORMÁTI...,83300,2995.0,12054.0,PORTARIA,723644,http://smaonline.rio.rj.gov.br/ConLegis/ato.as...
61862,0,DPT_19,2020-07-28,108111,557,2020-06-08,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre os programas de autocontrole (PAC...,268,31.0,33.0,"A SUBSECRETÁRIA DE VIGILÂNCIA, FISCALIZAÇÃO SA...",56226,2793.0,8432.0,PORTARIA,708087,http://smaonline.rio.rj.gov.br/ConLegis/ato.as...
61863,0,DPT_19,2020-07-28,108112,560,2020-06-08,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre os requisitos técnicos relativos ...,232,25.0,28.0,"A SUBSECRETÁRIA DE VIGILÂNCIA, FISCALIZAÇÃO SA...",64411,2626.0,9348.0,PORTARIA,708100,http://smaonline.rio.rj.gov.br/ConLegis/ato.as...


In [17]:
BATCH_SIZE = 15

<h3>Evaluate which regulators get in each department to make the intra-class balancing strategy</h3>

In [19]:
lt = ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']

In [20]:
for l in lt:
    dfr = data_relevant[data_relevant.department.isin([l])]
    dddd = dict(dfr.regulatory_authority.value_counts())
    print(l,' - ',list(dict(sorted(dddd.items(), key=lambda item: item[1], reverse=True)).items())[:6])

DPT_25  -  [('BACEN', 666), ('PRESIDÊNCIA_DA_REPÚBLICA', 29), ('COAF', 28), ('DOU', 26), ('ANAC', 14), ('RFB', 14)]
DPT_3  -  [('B3', 302), ('BACEN', 175), ('CETIP', 99), ('ANBIMA', 55), ('CVM', 49), ('RFB', 7)]
DPT_2  -  [('CVM', 253), ('ANBIMA', 221), ('B3', 147), ('COAF', 18), ('BACEN', 16), ('RFB', 13)]
DPT_9  -  [('BACEN', 205), ('CIP', 198), ('DOU', 64), ('BNDES', 55), ('PRESIDÊNCIA_DA_REPÚBLICA', 32), ('NUCLEA', 17)]
DPT_5  -  [('BACEN', 193), ('CVM', 126), ('DOU', 107), ('RFB', 79), ('CFC', 22), ('CPC', 13)]
DPT_16  -  [('B3', 97), ('CETIP', 85), ('BACEN', 68), ('STN', 67), ('RFB', 24), ('ANBIMA', 20)]


In [21]:
# We chose the regulators with more documents in the relevant class of each of the three departments.
# The regulators must exist in both classes of the department.
# Only regulators with more than 10 documents in the relevant class where chosen.
# To make the undersampling, we took the regulator with less documents in the relevant class for each department, 26 documents at DPT_25, for example.

# DPT_25  -  [('BACEN', 666), ('PRESIDÊNCIA_DA_REPÚBLICA', 29), ('COAF', 28), ('DOU', 26)] # relevant
# DPT_25  -  [('DOU', 2982), ('BACEN', 467), ('COAF', 48), ('PRESIDÊNCIA_DA_REPÚBLICA', 32)] # irrelevant

# DPT_3  -  [('B3', 302), ('BACEN', 175), ('ANBIMA', 55), ('CVM', 49)] # relevant
# DPT_3  -  [('B3', 592), ('CVM', 368), ('BACEN', 257), ('ANBIMA', 85)] # irrelevant

# DPT_2  -  [('CVM', 253), ('ANBIMA', 221), ('B3', 147), ('RFB', 13)] # relevant
# DPT_2  -  [('B3', 1921), ('CVM', 783), ('RFB', 678), ('ANBIMA', 399)] # irrelevant

In [22]:
for l in lt:
    dfi = data_irrelevant[data_irrelevant.department.isin([l])]
    dddd = dict(dfi.regulatory_authority.value_counts())
    print(l,' - ',list(dict(sorted(dddd.items(), key=lambda item: item[1], reverse=True)).items())[:6])

DPT_25  -  [('DOU', 2982), ('BACEN', 467), ('ANAC', 421), ('COAF', 48), ('PRESIDÊNCIA_DA_REPÚBLICA', 32), ('NUCLEA', 26)]
DPT_3  -  [('B3', 592), ('CVM', 368), ('STN', 264), ('BACEN', 257), ('RFB', 161), ('ANBIMA', 85)]
DPT_2  -  [('B3', 1921), ('CVM', 783), ('CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO', 744), ('RFB', 678), ('ANBIMA', 399), ('SUSEP', 391)]
DPT_9  -  [('BACEN', 826), ('CIP', 712), ('CVM', 531), ('RFB', 331), ('STN', 269), ('DOU', 230)]
DPT_5  -  [('DOU', 1147), ('BACEN', 658), ('RFB', 327), ('PRESIDÊNCIA_DA_REPÚBLICA', 201), ('CVM', 158), ('NUCLEA', 60)]
DPT_16  -  [('BACEN', 337), ('STN', 294), ('RFB', 270), ('CVM', 162), ('B3', 121), ('CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO', 101)]


In [23]:
aux_relevant = []

SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25 = 26
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3 = 49
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2 = 13

# DPT_25
dpt_25 = data_relevant[data_relevant.department.isin(['DPT_25'])]

regulator = dpt_25[dpt_25.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['PRESIDÊNCIA_DA_REPÚBLICA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['COAF'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['DOU'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)
# DPT_25

# DPT_3
dpt_3 = data_relevant[data_relevant.department.isin(['DPT_3'])]

regulator = dpt_3[dpt_3.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)
# DPT_3

# DPT_2
dpt_2 = data_relevant[data_relevant.department.isin(['DPT_2'])]

regulator = dpt_2[dpt_2.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['RFB'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)
# DPT_2

data_relevant = pd.concat(aux_relevant)

data_relevant[data_relevant.text_words < 50]

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url


In [24]:
data_relevant.shape

(352, 18)

In [25]:
(13*4)+(49*4)+(26*4)

352

In [26]:
aux_irrelevant = []

SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25 = 26
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3 = 49
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2 = 13

# DPT_25
dpt_25 = data_irrelevant[data_irrelevant.department.isin(['DPT_25'])]

regulator = dpt_25[dpt_25.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['PRESIDÊNCIA_DA_REPÚBLICA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['COAF'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = dpt_25[dpt_25.regulatory_authority.isin(['DOU'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)
# DPT_25

# DPT_3
dpt_3 = data_irrelevant[data_irrelevant.department.isin(['DPT_3'])]

regulator = dpt_3[dpt_3.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = dpt_3[dpt_3.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)
# DPT_3

# DPT_2
dpt_2 = data_irrelevant[data_irrelevant.department.isin(['DPT_2'])]

regulator = dpt_2[dpt_2.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = dpt_2[dpt_2.regulatory_authority.isin(['RFB'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)
# DPT_2

data_irrelevant = pd.concat(aux_irrelevant)

data_irrelevant[data_irrelevant.text_words < 50]

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url


In [27]:
data_irrelevant.shape

(352, 18)

In [28]:
data_irrelevant.head(3)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
7846,0,DPT_25,2020-08-31,2169,0301/2020,2020-08-31,BACEN,Caracterizado fornecimento intempestivo ao Ba...,294,36.0,49.0,​DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONAD...,758,81.0,113.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729080,https://www.bcb.gov.br/estabilidadefinanceira/...
7847,0,DPT_25,2020-08-31,2170,36121,2020-08-31,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,7080,407.0,966.0,COMUNICADO,729131,https://www.bcb.gov.br/estabilidadefinanceira/...
7848,0,DPT_25,2020-08-31,2171,36123,2020-08-31,BACEN,Divulga nome de pessoas com intenção de ocupar...,140,19.0,21.0,Divulgamos os nomes de pretendentes a cargos d...,1558,149.0,276.0,COMUNICADO,729151,https://www.bcb.gov.br/estabilidadefinanceira/...


In [29]:
data_relevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
url                     352
dtype: int64

In [30]:
data_relevant = data_relevant[data_relevant.text_words >= 50]
data_relevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
url                     352
dtype: int64

In [31]:
data_irrelevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
url                     352
dtype: int64

In [32]:
data_irrelevant = data_irrelevant[data_irrelevant.text_words >= 50]
data_irrelevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
url                     352
dtype: int64

In [33]:
data_relevant['department'].unique()

array(['DPT_25', 'DPT_3', 'DPT_2'], dtype=object)

In [34]:
len(list(data_relevant['department'].unique()))

3

In [35]:
class Model_BERT():
    
    def model_path_def(self, name_model):
        if name_model == "BERTIMBAU_LARGE":
            model_path = MODEL_PATH
        elif name_model == "XXXX":
            model_path = MODEL_PATH_NEW
        else:
            print("Nome de modelo inválido")
            model_path = "None"
            
        return model_path

    def tokenizer_model(self, model_path, df, token):
        tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

        # Tokenizing the training data
        encoded_data_train = tokenizer.batch_encode_plus(
            df[df.data_type=='train'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True, 
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )

        # Tokenizing test data
        encoded_data_val = tokenizer.batch_encode_plus(
            df[df.data_type=='val'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True,
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )


        input_ids_train = encoded_data_train['input_ids']
        attention_masks_train = encoded_data_train['attention_mask']
        labels_train = torch.tensor(df[df.data_type=='train'].label_num.values)

        input_ids_val = encoded_data_val['input_ids']
        attention_masks_val = encoded_data_val['attention_mask']
        labels_val = torch.tensor(df[df.data_type=='val'].label_num.values)

        dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
        dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
        
        return(dataset_train, dataset_val)

    def setup_model(self, model_path, dataset_train, dataset_val, epochs):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        batch_size = BATCH_SIZE
        dataloader_train = DataLoader(dataset_train, 
                                      sampler=RandomSampler(dataset_train), 
                                      batch_size=batch_size)
        dataloader_validation = DataLoader(dataset_val, 
                                           sampler=SequentialSampler(dataset_val), 
                                           batch_size=batch_size)
        optimizer = AdamW(model.parameters(),
                          lr=1e-5, 
                          eps=1e-8)

        scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps=100,
                                                    num_training_steps=len(dataloader_train)*epochs)
        return(model, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device)


    def f1_score_func(self, preds, labels):
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        scr = f1_score(labels_flat, preds_flat, average='binary')
        lg.info(f"f1_score_binary: {scr}")
        return scr

    def accuracy_per_class(self, preds, labels):
        label_dict_inverse = {v: k for k, v in LABEL_DICT.items()}

        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat==label]
            y_true = labels_flat[labels_flat==label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            print(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
            lg.info(f'Class: {label_dict_inverse[label]}')
            lg.info(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            lg.info(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
        
    def evaluate(self, model, dataloader_val, device):

        model.eval()

        loss_val_total = 0
        predictions, true_vals = [], []

        for batch in dataloader_val:

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            with torch.no_grad():        
                outputs = model(**inputs)

            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        loss_val_avg = loss_val_total/len(dataloader_val) 

        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)

        lg.info(f"loss_val_avg, predictions, true_vals: {loss_val_avg}, {predictions}, {true_vals}")

        return loss_val_avg, predictions, true_vals

    def results(self, model_path, model_name, epochs, dataloader_validation, device, token):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        model.to(device)

        model.load_state_dict(torch.load(f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epochs}.model', map_location=torch.device('cpu')))

        _, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
        Model_BERT().accuracy_per_class(predictions, true_vals)
        from sklearn import metrics
        preds_flat = np.argmax(predictions, axis=1).flatten()
        metrics_report = metrics.classification_report(true_vals,preds_flat, output_dict=True)
        lg.info(metrics_report)
        
        return metrics_report
    

    def training_model(self, model, model_name, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device, token):
        seed_val = 1
        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        epoch_array = []
        f1_array = []
        train_loss_array = []
        val_loss_array = []

        for epoch in tqdm(range(1, epochs+1)):

            model.train()

            loss_train_total = 0

            progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
            for batch in progress_bar:

                model.zero_grad()

                batch = tuple(b.to(device) for b in batch)

                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[2],
                         }       

                outputs = model(**inputs)

                loss = outputs[0]
                loss_train_total += loss.item()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()
                scheduler.step()

                progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


            torch.save(model.state_dict(),f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}.model')
            model.save_pretrained(f"{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}")
            tqdm.write(f'\nEpoch {epoch}')
            print(f'\nEpoch {epoch}')
            lg.info(f'\nEpoch {epoch}')

            loss_train_avg = loss_train_total/len(dataloader_train)            
            tqdm.write(f'Training loss: {loss_train_avg}')
            lg.info(f'Training loss: {loss_train_avg}')

            val_loss, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
            val_f1 = Model_BERT().f1_score_func(predictions, true_vals)
            tqdm.write(f'Validation loss: {val_loss}')
            print(f'Validation loss: {val_loss}')
            lg.info(f'Validation loss: {val_loss}')
            
            tqdm.write(f'F1 Score (Weighted/Binary): {val_f1}')
            print(f'F1 Score (Weighted/Binary): {val_f1}')
            lg.info(f'F1 Score (Weighted/Binary): {val_f1}')
            
            epoch_array.append(epoch)
            f1_array.append(val_f1)
            val_loss_array.append(val_loss)
            train_loss_array.append(loss_train_avg)

        return(epoch_array, f1_array, val_loss_array, train_loss_array)
    
    def read_df(self, df):
        df['label_num'] = df.label.replace(LABEL_DICT)

        X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label_num.values, 
                                                  test_size=0.4, 
                                                  random_state=42, 
                                                  stratify=df.label.values)
        df['data_type'] = ['not_set']*df.shape[0]
        df.loc[X_train, 'data_type'] = 'train'
        df.loc[X_val, 'data_type'] = 'val'
        
        return(df)

    def call_setup(self, model_name, df, token, epochs):
        df = Model_BERT().read_df(df)
        model_path = Model_BERT().model_path_def(model_name)
        print(model_path)
        dataset_train, dataset_val = Model_BERT().tokenizer_model(model_path, df, token)
        model, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device = Model_BERT().setup_model(model_path, dataset_train, dataset_val, epochs)
        epoch_array, f1_array, val_loss_array, train_loss_array = Model_BERT().training_model(model, model_name, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device, token)
        df_performa = pd.DataFrame()
        df_performa["epoch"] = epoch_array
        df_performa["f1"] = f1_array
        df_performa["val_loss"] = val_loss_array
        df_performa["train_loss"] = train_loss_array
        metrics_report = Model_BERT().results(model_path,model_name,epochs,dataloader_val, device, token)
        lg.info(f'Metric Report Call Setup: {metrics_report}')
        
        return metrics_report, df_performa

In [36]:
def cleaning(df,column):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    return df

<h3>Selecting the Departments</h3>

In [37]:
boards = data_relevant['department']
# Getting the 6 most populated boards
boards_list = list(boards.unique())[:DEPARTMENTS_QUANTITY_TO_TRAIN]
lg.info(f"boards_list: {boards_list}")

2024-03-01 18:32:25.981 | INFO     | __main__:<module>:4 - boards_list: ['DPT_25', 'DPT_3', 'DPT_2']


In [38]:
df_modelagem_init = pd.concat([data_irrelevant, data_relevant])

In [39]:
df_modelagem_init['class'] = df_modelagem_init['class'].astype(str)

In [40]:
df_modelagem_init = df_modelagem_init.rename(columns={"class":"label"})

In [41]:
df_modelagem_init.count()

label                   704
department              704
entry_date              704
general_id              704
normative_identifier    704
publication_date        704
regulatory_authority    704
subject                 704
subject_length          704
subject_unique_words    704
subject_words           704
text                    704
text_length             704
text_unique_words       704
text_words              704
type                    704
unique_document_id      704
url                     704
dtype: int64

In [ ]:
for department in boards_list:
    lg.debug(f"Department: {department}")
    df_modelagem = df_modelagem_init[df_modelagem_init.department == department]

    df_modelagem = df_modelagem.dropna()
    df_modelagem = df_modelagem.reset_index(drop=True)
    df_modelagem = df_modelagem.rename(columns={"text":"content"})

    df_modelagem = cleaning(df_modelagem,"content")

    new_content = []
    for texto in df_modelagem.content:
        new_content.append(texto.replace("[BUSINESS NEED]", ""))
    df_modelagem["new_content"] = new_content
    
    LABEL_DICT = {'0': 0, '1': 1}

    MODEL_SAVE_PATH = "notebooks/testes/scoring"
    BERTIMBAU_LARGE = "dataset/bert-large-portuguese-cased"
    BERTIMBAU_TOKENIZER = "tokenizer/BERTimbau_Tokenizer"

    MODEL_PATH = BERTIMBAU_LARGE
    MODEL_PATH_NEW = BERTIMBAU_LARGE

    tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

    lg.debug(f"Department count samples: {df_modelagem.count()}")
    lg.debug(f"Department count samples class 1: {df_modelagem[df_modelagem.label == '1'].count()}")
    lg.debug(f"Department count samples class 0: {df_modelagem[df_modelagem.label == '0'].count()}")
    

    Model_BERT().call_setup("BERTIMBAU_LARGE", df_modelagem, 512, 5)

<h2>The output result is in the next cell. It was copied from the log file.</h2>

In [ ]:
# 2023-10-17 08:59:36.446 | INFO     | __main__:<module>:5 - 

# ####################################### NEW EXECUTION #######################################


# 2023-10-17 08:59:36.447 | INFO     | __main__:<module>:6 - Processors (cores) available: 38
# 2023-10-17 08:59:46.524 | INFO     | __main__:<module>:4 - boards_list: ['DPT_25', 'DPT_3', 'DPT_2']
# 2023-10-17 08:59:46.571 | DEBUG    | __main__:<module>:2 - Department: DPT_25
# 2023-10-17 08:59:47.080 | DEBUG    | __main__:<module>:37 - Department count samples: label                   208
# department              208
# entry_date              208
# general_id              208
# normative_identifier    208
# publication_date        208
# regulatory_authority    208
# subject                 208
# subject_length          208
# subject_unique_words    208
# subject_words           208
# content                 208
# text_length             208
# text_unique_words       208
# text_words              208
# type                    208
# unique_document_id      208
# url                     208
# new_content             208
# dtype: int64
# 2023-10-17 08:59:47.083 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   104
# department              104
# entry_date              104
# general_id              104
# normative_identifier    104
# publication_date        104
# regulatory_authority    104
# subject                 104
# subject_length          104
# subject_unique_words    104
# subject_words           104
# content                 104
# text_length             104
# text_unique_words       104
# text_words              104
# type                    104
# unique_document_id      104
# url                     104
# new_content             104
# dtype: int64
# 2023-10-17 08:59:47.086 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   104
# department              104
# entry_date              104
# general_id              104
# normative_identifier    104
# publication_date        104
# regulatory_authority    104
# subject                 104
# subject_length          104
# subject_unique_words    104
# subject_words           104
# content                 104
# text_length             104
# text_unique_words       104
# text_words              104
# type                    104
# unique_document_id      104
# url                     104
# new_content             104
# dtype: int64
# 2023-10-17 09:00:26.088 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-17 09:00:26.090 | INFO     | __main__:training_model:214 - Training loss: 0.8070704672071669
# 2023-10-17 09:00:28.553 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.9369950046141943, [[ 0.11584277 -0.21372697]
 
# 2023-10-17 09:00:28.556 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.11538461538461536
# 2023-10-17 09:00:28.558 | INFO     | __main__:training_model:220 - Validation loss: 0.9369950046141943
# 2023-10-17 09:00:28.559 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.11538461538461536
# 2023-10-17 09:00:47.697 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-17 09:00:47.699 | INFO     | __main__:training_model:214 - Training loss: 0.7290287945005629
# 2023-10-17 09:00:50.152 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.7656629284222921, [[-0.15477064  0.3268677 ]
 
# 2023-10-17 09:00:50.156 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.24242424242424246
# 2023-10-17 09:00:50.157 | INFO     | __main__:training_model:220 - Validation loss: 0.7656629284222921
# 2023-10-17 09:00:50.158 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.24242424242424246
# 2023-10-17 09:01:09.105 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-17 09:01:09.107 | INFO     | __main__:training_model:214 - Training loss: 0.683078043990665
# 2023-10-17 09:01:11.603 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.7144554853439331, [[-0.16036744  0.38246924]
 
# 2023-10-17 09:01:11.607 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.4948453608247423
# 2023-10-17 09:01:11.608 | INFO     | __main__:training_model:220 - Validation loss: 0.7144554853439331
# 2023-10-17 09:01:11.609 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.4948453608247423
# 2023-10-17 09:01:30.801 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-17 09:01:30.802 | INFO     | __main__:training_model:214 - Training loss: 0.6523057421048483
# 2023-10-17 09:01:33.282 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.7454182902971903, [[-0.16628781  0.31541863]
 
# 2023-10-17 09:01:33.285 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.3636363636363636
# 2023-10-17 09:01:33.287 | INFO     | __main__:training_model:220 - Validation loss: 0.7454182902971903
# 2023-10-17 09:01:33.288 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.3636363636363636
# 2023-10-17 09:01:52.703 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-17 09:01:52.704 | INFO     | __main__:training_model:214 - Training loss: 0.5856314500172933
# 2023-10-17 09:01:55.186 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.7744867205619812, [[-0.29336476  0.11224861]
 
# 2023-10-17 09:01:55.190 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.4
# 2023-10-17 09:01:55.192 | INFO     | __main__:training_model:220 - Validation loss: 0.7744867205619812
# 2023-10-17 09:01:55.193 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.4
# 2023-10-17 09:02:02.207 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.7744867205619812, [[-0.29336476  0.11224861]
 
# 2023-10-17 09:02:02.209 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-17 09:02:02.210 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 24/42
# 2023-10-17 09:02:02.211 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.5714285714285714

# 2023-10-17 09:02:02.212 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-17 09:02:02.213 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 15/42
# 2023-10-17 09:02:02.214 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.35714285714285715

# 2023-10-17 09:02:02.221 | INFO     | __main__:results:157 - {'0': {'precision': 0.47058823529411764, 'recall': 0.5714285714285714, 'f1-score': 0.5161290322580646, 'support': 42}, '1': {'precision': 0.45454545454545453, 'recall': 0.35714285714285715, 'f1-score': 0.4, 'support': 42}, 'accuracy': 0.4642857142857143, 'macro avg': {'precision': 0.46256684491978606, 'recall': 0.4642857142857143, 'f1-score': 0.4580645161290323, 'support': 84}, 'weighted avg': {'precision': 0.4625668449197861, 'recall': 0.4642857142857143, 'f1-score': 0.4580645161290323, 'support': 84}}
# 2023-10-17 09:02:02.224 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.47058823529411764, 'recall': 0.5714285714285714, 'f1-score': 0.5161290322580646, 'support': 42}, '1': {'precision': 0.45454545454545453, 'recall': 0.35714285714285715, 'f1-score': 0.4, 'support': 42}, 'accuracy': 0.4642857142857143, 'macro avg': {'precision': 0.46256684491978606, 'recall': 0.4642857142857143, 'f1-score': 0.4580645161290323, 'support': 84}, 'weighted avg': {'precision': 0.4625668449197861, 'recall': 0.4642857142857143, 'f1-score': 0.4580645161290323, 'support': 84}}
# 2023-10-17 09:02:02.229 | DEBUG    | __main__:<module>:2 - Department: DPT_3
# 2023-10-17 09:02:05.143 | DEBUG    | __main__:<module>:37 - Department count samples: label                   392
# department              392
# entry_date              392
# general_id              392
# normative_identifier    392
# publication_date        392
# regulatory_authority    392
# subject                 392
# subject_length          392
# subject_unique_words    392
# subject_words           392
# content                 392
# text_length             392
# text_unique_words       392
# text_words              392
# type                    392
# unique_document_id      392
# url                     392
# new_content             392
# dtype: int64
# 2023-10-17 09:02:05.147 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   196
# department              196
# entry_date              196
# general_id              196
# normative_identifier    196
# publication_date        196
# regulatory_authority    196
# subject                 196
# subject_length          196
# subject_unique_words    196
# subject_words           196
# content                 196
# text_length             196
# text_unique_words       196
# text_words              196
# type                    196
# unique_document_id      196
# url                     196
# new_content             196
# dtype: int64
# 2023-10-17 09:02:05.150 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   196
# department              196
# entry_date              196
# general_id              196
# normative_identifier    196
# publication_date        196
# regulatory_authority    196
# subject                 196
# subject_length          196
# subject_unique_words    196
# subject_words           196
# content                 196
# text_length             196
# text_unique_words       196
# text_words              196
# type                    196
# unique_document_id      196
# url                     196
# new_content             196
# dtype: int64
# 2023-10-17 09:03:23.978 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-17 09:03:23.980 | INFO     | __main__:training_model:214 - Training loss: 0.7419208958745003
# 2023-10-17 09:03:28.579 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6986891248009421, [[ 2.18956433e-02  4.07977998e-02]
 
# 2023-10-17 09:03:28.582 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6439024390243901
# 2023-10-17 09:03:28.583 | INFO     | __main__:training_model:220 - Validation loss: 0.6986891248009421
# 2023-10-17 09:03:28.584 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6439024390243901
# 2023-10-17 09:03:57.781 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-17 09:03:57.783 | INFO     | __main__:training_model:214 - Training loss: 0.6896974667906761
# 2023-10-17 09:04:02.450 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6906430883841082, [[ 1.84297219e-01  6.52469024e-02]
 
# 2023-10-17 09:04:02.453 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.33070866141732286
# 2023-10-17 09:04:02.455 | INFO     | __main__:training_model:220 - Validation loss: 0.6906430883841082
# 2023-10-17 09:04:02.456 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.33070866141732286
# 2023-10-17 09:04:31.602 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-17 09:04:31.604 | INFO     | __main__:training_model:214 - Training loss: 0.6871334128081799
# 2023-10-17 09:04:36.234 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.676726904782382, [[ 0.4697189   0.5200419 ]
 
# 2023-10-17 09:04:36.238 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.5818181818181819
# 2023-10-17 09:04:36.240 | INFO     | __main__:training_model:220 - Validation loss: 0.676726904782382
# 2023-10-17 09:04:36.241 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.5818181818181819
# 2023-10-17 09:05:04.246 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-17 09:05:04.247 | INFO     | __main__:training_model:214 - Training loss: 0.6661775670945644
# 2023-10-17 09:05:08.905 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.666452941569415, [[ 4.09457505e-01  6.32930160e-01]
 
# 2023-10-17 09:05:08.909 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7090909090909091
# 2023-10-17 09:05:08.910 | INFO     | __main__:training_model:220 - Validation loss: 0.666452941569415
# 2023-10-17 09:05:08.911 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7090909090909091
# 2023-10-17 09:05:38.337 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-17 09:05:38.339 | INFO     | __main__:training_model:214 - Training loss: 0.6234667673707008
# 2023-10-17 09:05:42.975 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6464693139899861, [[ 0.468196    0.6222204 ]
 
# 2023-10-17 09:05:42.979 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6583850931677019
# 2023-10-17 09:05:42.981 | INFO     | __main__:training_model:220 - Validation loss: 0.6464693139899861
# 2023-10-17 09:05:42.982 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6583850931677019
# 2023-10-17 09:05:51.862 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6464693139899861, [[ 0.468196    0.6222204 ]
 
# 2023-10-17 09:05:51.864 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-17 09:05:51.865 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 49/79
# 2023-10-17 09:05:51.866 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.620253164556962

# 2023-10-17 09:05:51.867 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-17 09:05:51.868 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 53/78
# 2023-10-17 09:05:51.868 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6794871794871795

# 2023-10-17 09:05:51.876 | INFO     | __main__:results:157 - {'0': {'precision': 0.6621621621621622, 'recall': 0.620253164556962, 'f1-score': 0.6405228758169935, 'support': 79}, '1': {'precision': 0.6385542168674698, 'recall': 0.6794871794871795, 'f1-score': 0.6583850931677019, 'support': 78}, 'accuracy': 0.6496815286624203, 'macro avg': {'precision': 0.650358189514816, 'recall': 0.6498701720220708, 'f1-score': 0.6494539844923477, 'support': 157}, 'weighted avg': {'precision': 0.6504333740539711, 'recall': 0.6496815286624203, 'f1-score': 0.6493970984498295, 'support': 157}}
# 2023-10-17 09:05:51.878 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.6621621621621622, 'recall': 0.620253164556962, 'f1-score': 0.6405228758169935, 'support': 79}, '1': {'precision': 0.6385542168674698, 'recall': 0.6794871794871795, 'f1-score': 0.6583850931677019, 'support': 78}, 'accuracy': 0.6496815286624203, 'macro avg': {'precision': 0.650358189514816, 'recall': 0.6498701720220708, 'f1-score': 0.6494539844923477, 'support': 157}, 'weighted avg': {'precision': 0.6504333740539711, 'recall': 0.6496815286624203, 'f1-score': 0.6493970984498295, 'support': 157}}
# 2023-10-17 09:05:51.884 | DEBUG    | __main__:<module>:2 - Department: DPT_2
# 2023-10-17 09:05:52.203 | DEBUG    | __main__:<module>:37 - Department count samples: label                   104
# department              104
# entry_date              104
# general_id              104
# normative_identifier    104
# publication_date        104
# regulatory_authority    104
# subject                 104
# subject_length          104
# subject_unique_words    104
# subject_words           104
# content                 104
# text_length             104
# text_unique_words       104
# text_words              104
# type                    104
# unique_document_id      104
# url                     104
# new_content             104
# dtype: int64
# 2023-10-17 09:05:52.207 | DEBUG    | __main__:<module>:38 - Department count samples class 1: label                   52
# department              52
# entry_date              52
# general_id              52
# normative_identifier    52
# publication_date        52
# regulatory_authority    52
# subject                 52
# subject_length          52
# subject_unique_words    52
# subject_words           52
# content                 52
# text_length             52
# text_unique_words       52
# text_words              52
# type                    52
# unique_document_id      52
# url                     52
# new_content             52
# dtype: int64
# 2023-10-17 09:05:52.211 | DEBUG    | __main__:<module>:39 - Department count samples class 0: label                   52
# department              52
# entry_date              52
# general_id              52
# normative_identifier    52
# publication_date        52
# regulatory_authority    52
# subject                 52
# subject_length          52
# subject_unique_words    52
# subject_words           52
# content                 52
# text_length             52
# text_unique_words       52
# text_words              52
# type                    52
# unique_document_id      52
# url                     52
# new_content             52
# dtype: int64
# 2023-10-17 09:06:13.654 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-17 09:06:13.656 | INFO     | __main__:training_model:214 - Training loss: 0.737665057182312
# 2023-10-17 09:06:14.916 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6486289898554484, [[-1.0898372   0.0184208 ]
 
# 2023-10-17 09:06:14.919 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6774193548387097
# 2023-10-17 09:06:14.920 | INFO     | __main__:training_model:220 - Validation loss: 0.6486289898554484
# 2023-10-17 09:06:14.921 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6774193548387097
# 2023-10-17 09:06:28.168 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-17 09:06:28.170 | INFO     | __main__:training_model:214 - Training loss: 0.7567037224769593
# 2023-10-17 09:06:29.430 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6371599733829498, [[-0.94649523  0.00243429]
 
# 2023-10-17 09:06:29.433 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7000000000000001
# 2023-10-17 09:06:29.435 | INFO     | __main__:training_model:220 - Validation loss: 0.6371599733829498
# 2023-10-17 09:06:29.436 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7000000000000001
# 2023-10-17 09:06:42.923 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-17 09:06:42.925 | INFO     | __main__:training_model:214 - Training loss: 0.7088525414466857
# 2023-10-17 09:06:44.207 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6192876299222311, [[-0.5210849  -0.03820588]
 
# 2023-10-17 09:06:44.211 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7272727272727274
# 2023-10-17 09:06:44.212 | INFO     | __main__:training_model:220 - Validation loss: 0.6192876299222311
# 2023-10-17 09:06:44.213 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7272727272727274
# 2023-10-17 09:06:58.088 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-17 09:06:58.089 | INFO     | __main__:training_model:214 - Training loss: 0.7334847211837768
# 2023-10-17 09:06:59.349 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6167004704475403, [[-1.07914919e-03 -1.82628669e-02]
 
# 2023-10-17 09:06:59.352 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7450980392156863
# 2023-10-17 09:06:59.353 | INFO     | __main__:training_model:220 - Validation loss: 0.6167004704475403
# 2023-10-17 09:06:59.354 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7450980392156863
# 2023-10-17 09:07:13.117 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-17 09:07:13.118 | INFO     | __main__:training_model:214 - Training loss: 0.6873963236808777
# 2023-10-17 09:07:14.378 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.638961672782898, [[ 0.25843325  0.06933639]
 
# 2023-10-17 09:07:14.381 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6666666666666666
# 2023-10-17 09:07:14.382 | INFO     | __main__:training_model:220 - Validation loss: 0.638961672782898
# 2023-10-17 09:07:14.383 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6666666666666666
# 2023-10-17 09:07:19.659 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.638961672782898, [[ 0.25843325  0.06933639]
 
# 2023-10-17 09:07:19.661 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-17 09:07:19.661 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 14/21
# 2023-10-17 09:07:19.662 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6666666666666666

# 2023-10-17 09:07:19.663 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-17 09:07:19.664 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 14/21
# 2023-10-17 09:07:19.664 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.6666666666666666

# 2023-10-17 09:07:19.671 | INFO     | __main__:results:157 - {'0': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 21}, '1': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 21}, 'accuracy': 0.6666666666666666, 'macro avg': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 42}, 'weighted avg': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 42}}
# 2023-10-17 09:07:19.674 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 21}, '1': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 21}, 'accuracy': 0.6666666666666666, 'macro avg': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 42}, 'weighted avg': {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6666666666666666, 'support': 42}}
